In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
http://api.open-notify.org/iss-now.json



In [ ]:
pip install azure-eventhub

In [ ]:
pip install apscheduler

### Request

In [ ]:
import requests

response = requests.get("http://api.open-notify.org/iss-now.json")

# Print the status code of the response.
print(response.status_code)
print(response.content)

### Libraries


In [ ]:
##Libraries
import requests
from azure.eventhub import EventHubProducerClient, EventData
import time
from apscheduler.schedulers.blocking import BlockingScheduler
import json

### Data Streaming to the Event Streaming

In [ ]:


# Function to get ISS data and send it to Azure Event Hub
def send_iss_data():
    try:
        # Make a GET request to get the latest position of the ISS from the opennotify API
        response = requests.get("http://api.open-notify.org/iss-now.json")

        if response.status_code == 200:
            
            iss_data = response.json()

            # Convert iss_data to JSON string
            iss_data_json = json.dumps(iss_data)

            # Send data to Azure Event Hub
            eventhub_connection_str = (
                f"Endpoint=sb://esehwestuiy6nlrtxx94tjp0.servicebus.windows.net/;SharedAccessKeyName=key_d6f0cc67-af7e-a352-1d83-22e1128fff0d;SharedAccessKey=umWK1u7I/RQVWr3zuUFT106Qh6HuqNpG8+AEhKRsA/8=;EntityPath=es_fa31d11f-c702-4d57-9760-959d08651607"
                
            )
            
            producer = EventHubProducerClient.from_connection_string(eventhub_connection_str)

            with producer:
                event_data_batch = producer.create_batch()

                # Add the event data to the batch
                event_data_batch.add(EventData(body=iss_data_json))

                # Send the batch to the event hub
                producer.send_batch(event_data_batch)

            print("Data sent to Azure Event Hub successfully.")
        else:
            print(f"Failed to retrieve ISS data. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error: {str(e)}")

###https://learn.microsoft.com/en-us/fabric/real-time-analytics/event-streams/overview#various-source-connectors

# Set up a scheduler to run the function every 5 seconds
scheduler = BlockingScheduler()
scheduler.add_job(send_iss_data, 'interval', seconds=5)

try:
    scheduler.start()
except (KeyboardInterrupt, SystemExit):
    print("Scheduler stopped.")

###https://learn.microsoft.com/en-us/fabric/real-time-analytics/event-streams/overview#various-source-connectors

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_unixtime
import pandas as pd

# Read the Delta table
iss_positions_spark = spark.sql("SELECT * FROM LH_ES.ES_STREAM  LIMIT 1000")

# Show the DataFrame schema and data
iss_positions_spark.printSchema()
iss_positions_spark.show(truncate=False)

# Convert the timestamp to a human-readable format
iss_positions_spark = iss_positions_spark.withColumn("timestamp", from_unixtime(col("timestamp")))

# Extract latitude, longitude, and timestamp from the JSON column
iss_positions_spark = iss_positions_spark.withColumn("latitude", col("iss_position.latitude").cast("double"))
iss_positions_spark = iss_positions_spark.withColumn("longitude", col("iss_position.longitude").cast("double"))

# Select relevant columns
iss_position_df = iss_positions_spark.select("latitude", "longitude", "timestamp")

# Convert to Pandas DataFrame
iss_position_df_pandas = iss_position_df.toPandas()

# Show the Pandas DataFrame
print(iss_position_df_pandas)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming you have a DataFrame called iss_position_df_pandas with 'latitude', 'longitude', and 'timestamp' columns

# Create a scatter plot using Matplotlib
plt.scatter(iss_position_df_pandas['longitude'], iss_position_df_pandas['latitude'])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('ISS Positions Over Time')

# Annotate points with timestamps
for index, row in iss_position_df_pandas.iterrows():
    plt.annotate(row['timestamp'], (row['longitude'], row['latitude']), textcoords="offset points", xytext=(5,5), ha='left')

plt.show()
